# Bring in Data

In [2]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
import types
import pandas as pd
df = pd.read_csv("Event.csv")
df.head()

Opportunity         Time  Stage               Action
0       OP001  43456.60550      1       Emailed Client
1       OP001  43466.19231      1       Emailed Client
2       OP001  43471.51592      2  Meeting with Client
3       OP001  43479.39509      3  Presented to Client
4       OP001  43483.32009      3  Presented to Client

In [4]:
df['Event Key'] = df['Stage'].astype(str) +'-'+ df['Action']

In [5]:
events = list(df['Event Key'].unique())
ev = events.pop(8)
events.append(ev)
events = ['0-Dummy'] + events

In [6]:
events

['0-Dummy',
 '1-Emailed Client',
 '2-Meeting with Client',
 '3-Presented to Client',
 '4-Prepared Custom Demonstration',
 '5-Prepared POC',
 '6-Provided Pre-Sales Support',
 '8-Prepared Commercials',
 '9-Negotiation Meeting',
 '7-Prepared RFP Responses',
 '10-Commercials Signed',
 '11-Post Sales Support']

In [6]:
df.head()

Opportunity         Time  Stage               Action              Event Key
0       OP001  43456.60550      1       Emailed Client       1-Emailed Client
1       OP001  43466.19231      1       Emailed Client       1-Emailed Client
2       OP001  43471.51592      2  Meeting with Client  2-Meeting with Client
3       OP001  43479.39509      3  Presented to Client  3-Presented to Client
4       OP001  43483.32009      3  Presented to Client  3-Presented to Client

# Prepare Data

In [7]:
series_length = 5
categories = len(events)
time_steps = series_length -1 

In [8]:
df.set_index('Opportunity', inplace=True)

In [9]:
history = []
for op in df.index:
    for x in range(len(df['Stage'].loc[op].values)-series_length):
        history.append(df['Stage'].loc[op].values[x:x+series_length])

In [10]:
history[:5]

[array([1, 1, 2, 3, 3], dtype=int64),
 array([1, 2, 3, 3, 3], dtype=int64),
 array([2, 3, 3, 3, 4], dtype=int64),
 array([3, 3, 3, 4, 5], dtype=int64),
 array([3, 3, 4, 5, 5], dtype=int64)]

In [11]:
from tensorflow.keras.utils import to_categorical

In [12]:
X= np.array([row[:time_steps] for row in history])

In [13]:
X = to_categorical(X)

In [14]:
y=np.array([row[time_steps] for row in history])

In [15]:
y = to_categorical(y).reshape(-1,categories)

In [16]:
X.shape

(16176, 4, 12)

In [18]:
X

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]], dtype=float32)

In [17]:
y.shape

(16176, 12)

# Create Train and Test Set

In [19]:
from sklearn.model_selection import train_test_split

C:\Users\chandv15\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [21]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(11323, 4, 12) (4853, 4, 12) (11323, 12) (4853, 12)


# Modelling

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [23]:
model = Sequential()

In [24]:
model.add(LSTM(10, activation='relu', return_sequences=True, input_shape=(time_steps,categories)))
model.add(LSTM(10, activation='relu'))
model.add(Dense(categories, activation='softmax'))
model.compile('adam', loss='categorical_crossentropy')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 10)             920       
                                                                 
 lstm_1 (LSTM)               (None, 10)                840       
                                                                 
 dense (Dense)               (None, 12)                132       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(X_train,y_train, epochs=10)

Epoch 1/10
354/354 [==============================] - 7s 8ms/step - loss: 1.9098
Epoch 2/10
354/354 [==============================] - 3s 8ms/step - loss: 1.1868
Epoch 3/10
354/354 [==============================] - 3s 8ms/step - loss: 1.0850
Epoch 4/10
354/354 [==============================] - 3s 8ms/step - loss: 1.0376
Epoch 5/10
354/354 [==============================] - 3s 8ms/step - loss: 1.0083
Epoch 6/10
354/354 [==============================] - 3s 8ms/step - loss: 0.9891
Epoch 7/10
354/354 [==============================] - 3s 8ms/step - loss: 0.9782
Epoch 8/10
354/354 [==============================] - 3s 7ms/step - loss: 0.9690
Epoch 9/10
354/354 [==============================] - 3s 8ms/step - loss: 0.9616
Epoch 10/10
354/354 [==============================] - 3s 8ms/step - loss: 0.9536


# Generate Predictions

In [27]:
prediction = model.predict(X_test)

152/152 [==============================] - 1s 3ms/step


In [28]:
# Next event is
events[np.argmax(prediction[1])]

'4-Prepared Custom Demonstration'

In [29]:
np.argmax(y_test[1])

4

In [30]:
np.argmax(prediction[1])

4

# Evaluate Accuracy

In [31]:
import tensorflow as tf
from tensorflow.keras.metrics import top_k_categorical_accuracy

In [32]:
print('Correct event was in top prediction:', top_k_categorical_accuracy(y_test,prediction, k=1).numpy().sum()/top_k_categorical_accuracy(y_test,prediction, k=2).numpy().size)
print('Correct event was in top two prediction:', top_k_categorical_accuracy(y_test,prediction, k=2).numpy().sum()/top_k_categorical_accuracy(y_test,prediction, k=3).numpy().size)

Correct event was in top prediction: 0.5617144034617763
Correct event was in top two prediction: 0.9008860498660622


In [33]:
X[1]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)